# EpiMine Tutorial: Building Hierarchical Event Structures from Scene Graphs

This notebook demonstrates step-by-step how to build a hierarchical event structure from a video scene graph using the EpiMine methodology.

**EpiMine** is an unsupervised episode detection framework that detects episode boundaries through **co-occurrence pattern shifts** rather than semantic similarity alone.

## Section 1: Core Concepts

### What is EpiMine?

EpiMine (ACL 2025) is an unsupervised method for detecting episodes in sequential data. The key insight:

> **Episodes cannot be detected using semantic similarity alone—they require analyzing shifts in term co-occurrence patterns.**

### Core Philosophy

1. **Discriminative Key Terms**: Identify terms that distinguish foreground (current sample) from background (all samples)
2. **Co-occurrence as Structure Signal**: Terms that co-occur form "who-did-what" relationships; shifts indicate boundaries
3. **Statistical Thresholds**: Use `mean ± 1σ` to adaptively detect boundaries
4. **LLM Refinement**: Use LLM to generate human-readable episode descriptions

## Section 2: Setup and Imports

In [16]:
# pip install matplotlib and seaborn if not already installed
! pip install matplotlib seaborn

In [17]:
import json
import numpy as np
from pathlib import Path
from collections import Counter
from typing import Dict, List, Set, Tuple
from pprint import pprint

# For visualization
try:
    import matplotlib.pyplot as plt
    import seaborn as sns
    HAS_VIZ = True
except ImportError:
    HAS_VIZ = False
    print("matplotlib/seaborn not available - skipping visualizations")

print("Setup complete!")

Setup complete!


## Section 3: Load Sample Data

We'll load a single sample from the SGQA dataset to demonstrate the pipeline.

In [18]:
# Path to SGQA dataset
SGQA_PATH = Path("/home/jtu9/sgg/tsg-bench/resource/dataset/understanding/sgqa.jsonl")

# Load all samples
all_samples = []
with open(SGQA_PATH, 'r') as f:
    for line in f:
        if line.strip():
            all_samples.append(json.loads(line))

print(f"Loaded {len(all_samples)} samples")

# Use first sample for demonstration
sample = all_samples[0]
action_sequence = sample["context_graphs"]

print(f"\nSample ID: {sample['data_id']}")
print(f"Number of actions: {len(action_sequence)}")

Loaded 100 samples

Sample ID: 19cc4e42-39bb-41f9-b9de-9f2940eed6a2
Number of actions: 11


In [19]:
# Visualize the action sequence
print("=" * 80)
print("ACTION SEQUENCE (Scene Graphs)")
print("=" * 80)

def get_action_verb(action_graph):
    """Extract main verb from action graph."""
    for triplet in action_graph:
        if len(triplet) >= 3 and triplet[1] in {"verb", "verbs"}:
            return triplet[2]
    return "unknown"

for i, action_graph in enumerate(action_sequence):
    verb = get_action_verb(action_graph)
    print(f"\nAction {i} ({verb}):")
    for triplet in action_graph:
        print(f"  {triplet}")

ACTION SEQUENCE (Scene Graphs)

Action 0 (pick-up):
  ['pick-up', 'with', 'hand1']
  ['pick-up', 'with', 'hand2']
  ['mop-stick', 'from', 'floor']
  ['person', 'verb', 'pick-up']
  ['pick-up', 'dobj', 'mop-stick']

Action 1 (sweep):
  ['sweep', 'with', 'hand1']
  ['sweep', 'with', 'hand2']
  ['sweep', 'with', 'mop-stick']
  ['sweep', 'dobj', 'floor']
  ['sweep', 'in', 'car']
  ['person', 'verb', 'sweep']

Action 2 (close):
  ['close', 'with', 'hand1']
  ['close', 'dobj', 'door']
  ['person', 'verb', 'close']

Action 3 (place):
  ['place', 'with', 'hand2']
  ['place', 'dobj', 'mop-stick']
  ['place', 'on', 'floor']
  ['person', 'verb', 'place']

Action 4 (open):
  ['open', 'dobj', 'door']
  ['open', 'with', 'hand2']
  ['person', 'verb', 'open']

Action 5 (put):
  ['put', 'dobj', 'cloth']
  ['put', 'inside', 'car']
  ['put', 'with', 'hand1']
  ['person', 'verb', 'put']

Action 6 (move):
  ['person', 'verb', 'move']
  ['move', 'to', 'cabinet']

Action 7 (open):
  ['open', 'dobj', 'cabinet

## Section 4: Build Background Dataset

The **background dataset** contains all actions from the entire SGQA dataset. This provides the "general distribution" against which we can identify discriminative terms.

**Why do we need background?**
- Terms that appear frequently in both foreground AND background are not discriminative (e.g., "person")
- Terms that appear frequently in foreground but rarely in background ARE discriminative (e.g., "mop-stick")

In [4]:
def extract_terms_from_action(action_graph: List[List[str]]) -> Set[str]:
    """Extract all terms from an action graph."""
    terms = set()
    for triplet in action_graph:
        for term in triplet:
            terms.add(term)
    return terms

# Build background: all actions from all samples
background_actions = []
for s in all_samples:
    background_actions.extend(s["context_graphs"])

print(f"Background dataset: {len(background_actions)} action graphs")

# Count term frequencies in background
term_counts_bg = Counter()
for action_graph in background_actions:
    terms = extract_terms_from_action(action_graph)
    term_counts_bg.update(terms)

print(f"\nUnique terms in background: {len(term_counts_bg)}")
print(f"\nTop 10 most common terms:")
for term, count in term_counts_bg.most_common(10):
    print(f"  {term}: {count}")

Background dataset: 2546 action graphs

Unique terms in background: 560

Top 10 most common terms:
  person: 2545
  verb: 2527
  dobj: 2525
  with: 2446
  hand1: 2037
  hand2: 1088
  pick-up: 596
  on: 423
  from: 307
  place: 304


## Section 5: Understanding Key Parameters

### 5.1 What is `min_freq`?

**`min_freq`** is the minimum number of times a term must appear in the foreground (current sample) to be considered as a key term.

| Implementation | min_freq | Reason |
|---------------|----------|--------|
| Original EpiMine (News) | 5 | Large vocabulary, many rare words |
| SGQA Adaptation | 2 | Small, fixed vocabulary |

**Why filter by min_freq?**
- Prevents rare/one-off terms from affecting episode detection
- Ensures only **frequent** terms are considered as key terms
- Terms appearing ≤ min_freq times return salience = -1 (filtered out)

In [5]:
# Demonstrate min_freq effect
foreground = action_sequence  # Current sample's actions

# Count term frequencies in foreground
term_counts_fg = Counter()
for action_graph in foreground:
    terms = extract_terms_from_action(action_graph)
    term_counts_fg.update(terms)

print("Term frequencies in FOREGROUND (current sample):")
print("=" * 50)
for term, count in sorted(term_counts_fg.items(), key=lambda x: -x[1]):
    status = "✓ KEPT" if count > 2 else "✗ FILTERED (≤ min_freq=2)"
    print(f"  {term}: {count} → {status}")

Term frequencies in FOREGROUND (current sample):
  person: 11 → ✓ KEPT
  verb: 11 → ✓ KEPT
  with: 10 → ✓ KEPT
  dobj: 9 → ✓ KEPT
  hand1: 7 → ✓ KEPT
  hand2: 5 → ✓ KEPT
  mop-stick: 4 → ✓ KEPT
  floor: 3 → ✓ KEPT
  cloth: 3 → ✓ KEPT
  pick-up: 2 → ✗ FILTERED (≤ min_freq=2)
  from: 2 → ✗ FILTERED (≤ min_freq=2)
  car: 2 → ✗ FILTERED (≤ min_freq=2)
  in: 2 → ✗ FILTERED (≤ min_freq=2)
  door: 2 → ✗ FILTERED (≤ min_freq=2)
  open: 2 → ✗ FILTERED (≤ min_freq=2)
  to: 2 → ✗ FILTERED (≤ min_freq=2)
  cabinet: 2 → ✗ FILTERED (≤ min_freq=2)
  move: 2 → ✗ FILTERED (≤ min_freq=2)
  wall: 2 → ✗ FILTERED (≤ min_freq=2)
  sweep: 1 → ✗ FILTERED (≤ min_freq=2)
  close: 1 → ✗ FILTERED (≤ min_freq=2)
  place: 1 → ✗ FILTERED (≤ min_freq=2)
  on: 1 → ✗ FILTERED (≤ min_freq=2)
  put: 1 → ✗ FILTERED (≤ min_freq=2)
  inside: 1 → ✗ FILTERED (≤ min_freq=2)
  pick: 1 → ✗ FILTERED (≤ min_freq=2)


### 5.2 What is Term Expansion (cosine ≥ 0.75)?

**Term Expansion** is a technique used in the **original EpiMine** (news domain) to handle synonyms and semantically similar words.

#### The Problem (in News Domain)
News articles use varied vocabulary:
- "killed", "murdered", "slain", "assassinated" → same concept
- Without expansion, these are treated as separate terms

#### The Solution: Two-Stage Expansion

```
Stage 1: Expand key terms (threshold = 0.75)
─────────────────────────────────────────────
1. Compute salience for all vocabulary words
2. Keep high-salience words as "key terms"
3. For each excluded word:
   - Compute cosine similarity to all key terms
   - If max similarity ≥ 0.75, add to key terms

Stage 2: Cluster similar terms (threshold = 0.85)
─────────────────────────────────────────────────
1. Compute pairwise cosine similarity between key terms
2. Cluster terms with similarity ≥ 0.85
3. Treat clustered terms as ONE concept for co-occurrence
```

#### Why SGQA Doesn't Need Term Expansion

| Aspect | News Domain | SGQA Scene Graphs |
|--------|-------------|-------------------|
| Vocabulary | Open-ended (any word) | Fixed (predefined verbs, objects) |
| Synonyms | Common (killed/murdered) | None (actions are standardized) |
| Term matching | Fuzzy (need expansion) | Exact (triplet terms) |

**Scene graphs use a controlled vocabulary** — there's only one way to say "pick-up", so no expansion is needed.

In [6]:
# Illustrate what term expansion would do (conceptually)
print("TERM EXPANSION EXAMPLE (Original EpiMine - News Domain)")
print("=" * 60)
print()
print("Key terms identified: ['killed', 'attack', 'explosion']")
print()
print("Excluded terms with similarity to 'killed':")
print("  murdered  → cosine = 0.82 ≥ 0.75 → ADD to key terms ✓")
print("  slain     → cosine = 0.78 ≥ 0.75 → ADD to key terms ✓")
print("  injured   → cosine = 0.61 < 0.75 → SKIP ✗")
print()
print("After clustering (threshold = 0.85):")
print("  Cluster 0: ['killed', 'murdered', 'slain'] → treated as ONE concept")
print("  Cluster 1: ['attack']")
print("  Cluster 2: ['explosion']")
print()
print("-" * 60)
print("SGQA: No expansion needed (fixed vocabulary, no synonyms)")

TERM EXPANSION EXAMPLE (Original EpiMine - News Domain)

Key terms identified: ['killed', 'attack', 'explosion']

Excluded terms with similarity to 'killed':
  murdered  → cosine = 0.82 ≥ 0.75 → ADD to key terms ✓
  slain     → cosine = 0.78 ≥ 0.75 → ADD to key terms ✓
  injured   → cosine = 0.61 < 0.75 → SKIP ✗

After clustering (threshold = 0.85):
  Cluster 0: ['killed', 'murdered', 'slain'] → treated as ONE concept
  Cluster 1: ['attack']
  Cluster 2: ['explosion']

------------------------------------------------------------
SGQA: No expansion needed (fixed vocabulary, no synonyms)


## Section 6: Compute Salience

**Salience** measures how discriminative a term is for the current sample (foreground) compared to the general distribution (background).

### Formula
```
salience(term) = (1 + log(fg_count)²) × log(bg_total / bg_count)
```

- **`(1 + log(fg_count)²)`**: Boost terms that appear repeatedly in foreground
- **`log(bg_total / bg_count)`**: IDF component — penalize common terms

In [7]:
def compute_salience(term: str, foreground: List, term_counts_bg: Counter, 
                     num_bg: int, min_freq: int = 2) -> float:
    """
    Compute discriminative salience for a term.
    
    Args:
        term: The term to compute salience for
        foreground: List of action graphs (current sample)
        term_counts_bg: Background term counts
        num_bg: Total number of background actions
        min_freq: Minimum foreground frequency (default: 2)
    
    Returns:
        Salience score (higher = more discriminative), or -1 if filtered
    """
    # Count foreground occurrences
    fg_count = sum(1 for ag in foreground if term in extract_terms_from_action(ag))
    bg_count = term_counts_bg.get(term, 0)
    
    # Filter by min_freq
    if fg_count < min_freq:
        return -1.0
    
    # Compute salience
    if bg_count > 0:
        return (1 + np.log(fg_count) ** 2) * np.log(num_bg / bg_count)
    else:
        return (1 + np.log(fg_count) ** 2) * np.log(num_bg)

# Compute salience for all terms in foreground
num_bg = len(background_actions)
all_terms = set()
for ag in foreground:
    all_terms.update(extract_terms_from_action(ag))

term_salience = []
for term in all_terms:
    salience = compute_salience(term, foreground, term_counts_bg, num_bg, min_freq=2)
    term_salience.append((term, salience))

# Sort by salience (descending)
term_salience.sort(key=lambda x: -x[1])

print("SALIENCE SCORES")
print("=" * 60)
print(f"{'Term':<20} {'FG Count':<10} {'BG Count':<10} {'Salience':<10}")
print("-" * 60)
for term, salience in term_salience:
    fg_count = sum(1 for ag in foreground if term in extract_terms_from_action(ag))
    bg_count = term_counts_bg.get(term, 0)
    status = f"{salience:.2f}" if salience > 0 else "FILTERED"
    print(f"{term:<20} {fg_count:<10} {bg_count:<10} {status:<10}")

SALIENCE SCORES
Term                 FG Count   BG Count   Salience  
------------------------------------------------------------
mop-stick            4          4          18.86     
wall                 2          3          9.98      
floor                3          30         9.80      
door                 2          8          8.53      
cloth                3          64         8.13      
cabinet              2          14         7.70      
car                  2          38         6.22      
move                 2          40         6.15      
open                 2          47         5.91      
to                   2          81         5.10      
in                   2          99         4.81      
from                 2          307        3.13      
hand2                5          1088       3.05      
pick-up              2          596        2.15      
hand1                7          2037       1.07      
with                 10         2446       0.25      
verb 

## Section 7: Extract Key Terms

Key terms are the discriminative terms that will be used to detect episode boundaries.

In [8]:
# Filter to keep only positive salience terms
key_terms = [(t, s) for t, s in term_salience if s > 0]

print(f"KEY TERMS (sorted by salience)")
print("=" * 40)
for term, salience in key_terms:
    print(f"  {term}: {salience:.2f}")

print(f"\nTotal: {len(key_terms)} key terms")
print(f"Filtered: {len(term_salience) - len(key_terms)} terms (min_freq or low salience)")

KEY TERMS (sorted by salience)
  mop-stick: 18.86
  wall: 9.98
  floor: 9.80
  door: 8.53
  cloth: 8.13
  cabinet: 7.70
  car: 6.22
  move: 6.15
  open: 5.91
  to: 5.10
  in: 4.81
  from: 3.13
  hand2: 3.05
  pick-up: 2.15
  hand1: 1.07
  with: 0.25
  verb: 0.05
  dobj: 0.05
  person: 0.00

Total: 19 key terms
Filtered: 7 terms (min_freq or low salience)


## Section 8: Build Co-occurrence Matrix

The co-occurrence matrix measures similarity between actions based on shared key terms.

**Jaccard Similarity**: `|intersection| / |union|`

In [9]:
def compute_cooccurrence_matrix(action_sequence: List, key_terms: List[str]) -> np.ndarray:
    """
    Build co-occurrence matrix using Jaccard similarity.
    """
    num_actions = len(action_sequence)
    
    # Build term presence matrix
    term_to_idx = {t: i for i, t in enumerate(key_terms)}
    term_presence = np.zeros((num_actions, len(key_terms)))
    
    for action_idx, action_graph in enumerate(action_sequence):
        terms = extract_terms_from_action(action_graph)
        for term in terms:
            if term in term_to_idx:
                term_presence[action_idx, term_to_idx[term]] = 1
    
    # Compute Jaccard similarity between all pairs
    cooccur_matrix = np.zeros((num_actions, num_actions))
    
    for i in range(num_actions):
        for j in range(num_actions):
            if i != j:
                shared = np.sum(term_presence[i] * term_presence[j])
                total = np.sum(np.logical_or(term_presence[i], term_presence[j]))
                if total > 0:
                    cooccur_matrix[i, j] = shared / total
    
    return cooccur_matrix, term_presence

# Compute co-occurrence matrix
key_term_names = [t for t, _ in key_terms]
cooccur_matrix, term_presence = compute_cooccurrence_matrix(action_sequence, key_term_names)

print("CO-OCCURRENCE MATRIX (Jaccard Similarity)")
print("=" * 60)
print(f"Shape: {cooccur_matrix.shape}")
print()

# Print matrix
print("     ", end="")
for i in range(len(action_sequence)):
    print(f"A{i:<4}", end="")
print()

for i in range(len(action_sequence)):
    print(f"A{i}   ", end="")
    for j in range(len(action_sequence)):
        if i == j:
            print("  -  ", end="")
        else:
            print(f"{cooccur_matrix[i,j]:.2f} ", end="")
    print()

CO-OCCURRENCE MATRIX (Jaccard Similarity)
Shape: (11, 11)

     A0   A1   A2   A3   A4   A5   A6   A7   A8   A9   A10  
A0     -  0.67 0.45 0.70 0.42 0.42 0.15 0.42 0.55 0.27 0.64 
A1   0.67   -  0.45 0.70 0.42 0.55 0.15 0.42 0.42 0.36 0.50 
A2   0.45 0.45   -  0.44 0.62 0.62 0.22 0.62 0.62 0.36 0.40 
A3   0.70 0.70 0.44   -  0.56 0.40 0.20 0.40 0.40 0.23 0.67 
A4   0.42 0.42 0.62 0.56   -  0.40 0.20 0.56 0.40 0.23 0.50 
A5   0.42 0.55 0.62 0.40 0.40   -  0.20 0.56 0.75 0.45 0.36 
A6   0.15 0.15 0.22 0.20 0.20 0.20   -  0.33 0.20 0.40 0.18 
A7   0.42 0.42 0.62 0.40 0.56 0.56 0.33   -  0.56 0.33 0.36 
A8   0.55 0.42 0.62 0.40 0.40 0.75 0.20 0.56   -  0.45 0.36 
A9   0.27 0.36 0.36 0.23 0.23 0.45 0.40 0.33 0.45   -  0.31 
A10   0.64 0.50 0.40 0.67 0.50 0.36 0.18 0.36 0.36 0.31   -  


In [10]:
# Visualize as heatmap (if matplotlib available)
if HAS_VIZ:
    fig, ax = plt.subplots(figsize=(10, 8))
    
    # Create labels
    labels = [f"A{i}: {get_action_verb(ag)}" for i, ag in enumerate(action_sequence)]
    
    sns.heatmap(cooccur_matrix, annot=True, fmt=".2f", cmap="YlOrRd",
                xticklabels=labels, yticklabels=labels, ax=ax)
    ax.set_title("Co-occurrence Matrix (Jaccard Similarity)\nHigher = More Similar")
    plt.tight_layout()
    plt.show()
else:
    print("(Visualization skipped - matplotlib not available)")

(Visualization skipped - matplotlib not available)


## Section 9: Detect Episode Boundaries

Episode boundaries are detected when **consecutive co-occurrence drops below threshold**.

### Boundary Detection Logic
```
1. Compute consecutive co-occurrence scores: cooccur[i, i+1]
2. Calculate threshold: mean - 1×std
3. If score < threshold → START NEW EPISODE
```

In [11]:
def detect_episode_boundaries(cooccur_matrix: np.ndarray, threshold_std: float = 1.0) -> List[List[int]]:
    """
    Detect episode boundaries based on co-occurrence shifts.
    
    Args:
        cooccur_matrix: Pairwise Jaccard similarity matrix
        threshold_std: Standard deviations below mean for boundary
    
    Returns:
        List of episode groups (each is a list of action indices)
    """
    num_actions = cooccur_matrix.shape[0]
    
    if num_actions <= 1:
        return [[i for i in range(num_actions)]]
    
    # Compute consecutive co-occurrence scores
    consecutive_scores = []
    for i in range(num_actions - 1):
        consecutive_scores.append(cooccur_matrix[i, i + 1])
    
    print("CONSECUTIVE CO-OCCURRENCE SCORES")
    print("=" * 50)
    for i, score in enumerate(consecutive_scores):
        print(f"  A{i} → A{i+1}: {score:.3f}")
    
    # Compute threshold
    mean_score = np.mean(consecutive_scores)
    std_score = np.std(consecutive_scores)
    threshold = mean_score - threshold_std * std_score
    
    print(f"\nThreshold Calculation:")
    print(f"  Mean: {mean_score:.3f}")
    print(f"  Std:  {std_score:.3f}")
    print(f"  Threshold (mean - {threshold_std}σ): {threshold:.3f}")
    
    # Detect boundaries
    episodes = [[0]]
    print(f"\nBoundary Detection:")
    
    for i, score in enumerate(consecutive_scores):
        if score < threshold:
            print(f"  A{i} → A{i+1}: {score:.3f} < {threshold:.3f} → BOUNDARY DETECTED!")
            episodes.append([i + 1])
        else:
            print(f"  A{i} → A{i+1}: {score:.3f} ≥ {threshold:.3f} → continue episode")
            episodes[-1].append(i + 1)
    
    return episodes

# Detect boundaries
episode_boundaries = detect_episode_boundaries(cooccur_matrix, threshold_std=1.0)

print(f"\n" + "=" * 50)
print(f"DETECTED EPISODES: {len(episode_boundaries)}")
print("=" * 50)
for i, indices in enumerate(episode_boundaries):
    verbs = [get_action_verb(action_sequence[idx]) for idx in indices]
    print(f"\nEpisode {i}: Actions {indices}")
    print(f"  Verbs: {verbs}")

CONSECUTIVE CO-OCCURRENCE SCORES
  A0 → A1: 0.667
  A1 → A2: 0.455
  A2 → A3: 0.444
  A3 → A4: 0.556
  A4 → A5: 0.400
  A5 → A6: 0.200
  A6 → A7: 0.333
  A7 → A8: 0.556
  A8 → A9: 0.455
  A9 → A10: 0.308

Threshold Calculation:
  Mean: 0.437
  Std:  0.129
  Threshold (mean - 1.0σ): 0.308

Boundary Detection:
  A0 → A1: 0.667 ≥ 0.308 → continue episode
  A1 → A2: 0.455 ≥ 0.308 → continue episode
  A2 → A3: 0.444 ≥ 0.308 → continue episode
  A3 → A4: 0.556 ≥ 0.308 → continue episode
  A4 → A5: 0.400 ≥ 0.308 → continue episode
  A5 → A6: 0.200 < 0.308 → BOUNDARY DETECTED!
  A6 → A7: 0.333 ≥ 0.308 → continue episode
  A7 → A8: 0.556 ≥ 0.308 → continue episode
  A8 → A9: 0.455 ≥ 0.308 → continue episode
  A9 → A10: 0.308 < 0.308 → BOUNDARY DETECTED!

DETECTED EPISODES: 3

Episode 0: Actions [0, 1, 2, 3, 4, 5]
  Verbs: ['pick-up', 'sweep', 'close', 'place', 'open', 'put']

Episode 1: Actions [6, 7, 8, 9]
  Verbs: ['move', 'open', 'pick-up', 'move']

Episode 2: Actions [10]
  Verbs: ['pick']


## Section 10: Build Structured Episodes

Now we convert the detected boundaries into rich, structured episodes.

In [12]:
# Relation types for structured extraction
VERB_RELATIONS = {"verb", "verbs"}
OBJECT_RELATIONS = {"dobj", "obj", "pobj"}
INSTRUMENT_RELATIONS = {"with"}
SOURCE_RELATIONS = {"from"}
TARGET_RELATIONS = {"to", "on", "in", "into", "onto", "inside"}

def extract_structured_info(action_graph: List[List[str]]) -> Dict:
    """Extract structured information from an action graph."""
    info = {
        "agent": None,
        "action": None,
        "objects": [],
        "instruments": [],
        "source_locations": [],
        "target_locations": [],
    }
    
    for triplet in action_graph:
        if len(triplet) < 3:
            continue
        
        node1, relation, node2 = triplet[0], triplet[1], triplet[2]
        
        if relation in VERB_RELATIONS and node1 == "person":
            info["agent"] = node1
            info["action"] = node2
        elif relation in OBJECT_RELATIONS:
            info["objects"].append(node2)
        elif relation in INSTRUMENT_RELATIONS:
            info["instruments"].append(node2)
        elif relation in SOURCE_RELATIONS:
            info["source_locations"].append(node2)
        elif relation in TARGET_RELATIONS:
            info["target_locations"].append(node2)
    
    return info

def build_structured_episode(episode_id: int, action_indices: List[int], 
                              action_sequence: List, key_terms: List[Tuple[str, float]],
                              total_episodes: int) -> Dict:
    """Build a structured episode from action indices."""
    
    # Aggregate info from all actions
    agent = "person"
    primary_actions = []
    primary_objects = []
    instruments = []
    source_locations = []
    target_locations = []
    
    for idx in action_indices:
        info = extract_structured_info(action_sequence[idx])
        if info["agent"]:
            agent = info["agent"]
        if info["action"]:
            primary_actions.append(info["action"])
        primary_objects.extend(info["objects"])
        instruments.extend(info["instruments"])
        source_locations.extend(info["source_locations"])
        target_locations.extend(info["target_locations"])
    
    # Deduplicate
    primary_actions = list(dict.fromkeys(primary_actions))
    primary_objects = list(dict.fromkeys(primary_objects))
    instruments = list(dict.fromkeys(instruments))
    source_locations = list(dict.fromkeys(source_locations))
    target_locations = list(dict.fromkeys(target_locations))
    
    # Temporal position
    if episode_id == 0:
        position = "beginning"
    elif episode_id == total_episodes - 1:
        position = "end"
    else:
        position = "middle"
    
    # Get discriminative terms for this episode
    episode_terms = set()
    for idx in action_indices:
        episode_terms.update(extract_terms_from_action(action_sequence[idx]))
    discriminative_terms = [t for t, _ in key_terms if t in episode_terms][:5]
    
    # Build episode
    episode = {
        "episode_id": episode_id,
        "name": f"Episode {episode_id}",  # Will be refined by LLM
        "description": f"Actions: {', '.join(primary_actions)}",
        
        "core_structure": {
            "agent": agent,
            "primary_actions": primary_actions,
            "primary_objects": primary_objects,
            "instruments": instruments,
            "source_locations": source_locations if source_locations else None,
            "target_locations": target_locations if target_locations else None,
        },
        
        "time": {
            "action_indices": action_indices,
            "start_index": min(action_indices),
            "end_index": max(action_indices),
            "duration": len(action_indices),
        },
        
        "temporal_context": {
            "position": position,
            "precedes_episodes": list(range(episode_id + 1, total_episodes)) if episode_id < total_episodes - 1 else None,
            "follows_episodes": list(range(episode_id)) if episode_id > 0 else None,
        },
        
        "discriminative_terms": discriminative_terms,
    }
    
    return episode

# Build structured episodes
structured_episodes = []
for i, action_indices in enumerate(episode_boundaries):
    episode = build_structured_episode(
        episode_id=i,
        action_indices=action_indices,
        action_sequence=action_sequence,
        key_terms=key_terms,
        total_episodes=len(episode_boundaries)
    )
    structured_episodes.append(episode)

print("STRUCTURED EPISODES")
print("=" * 60)
for episode in structured_episodes:
    print(f"\n### Episode {episode['episode_id']}: {episode['name']}")
    print(f"Description: {episode['description']}")
    print(f"Actions: {episode['time']['action_indices']}")
    print(f"Core structure:")
    pprint(episode['core_structure'], indent=4)
    print(f"Discriminative terms: {episode['discriminative_terms']}")

STRUCTURED EPISODES

### Episode 0: Episode 0
Description: Actions: pick-up, sweep, close, place, open, put
Actions: [0, 1, 2, 3, 4, 5]
Core structure:
{   'agent': 'person',
    'instruments': ['hand1', 'hand2', 'mop-stick'],
    'primary_actions': ['pick-up', 'sweep', 'close', 'place', 'open', 'put'],
    'primary_objects': ['mop-stick', 'floor', 'door', 'cloth'],
    'source_locations': ['floor'],
    'target_locations': ['car', 'floor']}
Discriminative terms: ['mop-stick', 'floor', 'door', 'cloth', 'car']

### Episode 1: Episode 1
Description: Actions: move, open, pick-up
Actions: [6, 7, 8, 9]
Core structure:
{   'agent': 'person',
    'instruments': ['hand1'],
    'primary_actions': ['move', 'open', 'pick-up'],
    'primary_objects': ['cabinet', 'cloth'],
    'source_locations': None,
    'target_locations': ['cabinet', 'wall', 'cloth']}
Discriminative terms: ['wall', 'cloth', 'cabinet', 'move', 'open']

### Episode 2: Episode 2
Description: Actions: pick
Actions: [10]
Core struct

## Section 11: Final Hierarchical Structure

The complete hierarchical event structure ready for QA.

In [13]:
# Build final hierarchy
hierarchy = {
    "overall_goal": "Activity sequence",  # Would be refined by LLM
    "episodes": structured_episodes,
    "metadata": {
        "sample_id": sample["data_id"],
        "total_actions": len(action_sequence),
        "total_episodes": len(structured_episodes),
        "method": "EpiMine (co-occurrence boundary detection)"
    }
}

print("FINAL HIERARCHICAL EVENT STRUCTURE")
print("=" * 70)
print(json.dumps(hierarchy, indent=2))

FINAL HIERARCHICAL EVENT STRUCTURE
{
  "overall_goal": "Activity sequence",
  "episodes": [
    {
      "episode_id": 0,
      "name": "Episode 0",
      "description": "Actions: pick-up, sweep, close, place, open, put",
      "core_structure": {
        "agent": "person",
        "primary_actions": [
          "pick-up",
          "sweep",
          "close",
          "place",
          "open",
          "put"
        ],
        "primary_objects": [
          "mop-stick",
          "floor",
          "door",
          "cloth"
        ],
        "instruments": [
          "hand1",
          "hand2",
          "mop-stick"
        ],
        "source_locations": [
          "floor"
        ],
        "target_locations": [
          "car",
          "floor"
        ]
      },
      "time": {
        "action_indices": [
          0,
          1,
          2,
          3,
          4,
          5
        ],
        "start_index": 0,
        "end_index": 5,
        "duration": 6
      },
    

## Section 12: Visualize Timeline

Format the hierarchy for QA prompts.

In [14]:
def format_timeline(hierarchy: Dict, action_sequence: List) -> str:
    """Format hierarchy as a readable timeline."""
    lines = []
    lines.append(f"## Overall Goal\n{hierarchy['overall_goal']}\n")
    lines.append("## Activity Timeline\n")
    
    for episode in hierarchy["episodes"]:
        ep_id = episode["episode_id"]
        name = episode["name"]
        desc = episode["description"]
        core = episode["core_structure"]
        time_info = episode["time"]
        disc_terms = episode.get("discriminative_terms", [])
        
        lines.append(f"### Episode {ep_id}: {name}")
        lines.append(f"{desc}\n")
        
        lines.append("**Structure:**")
        lines.append(f"- Agent: {core['agent']}")
        lines.append(f"- Actions: {', '.join(core['primary_actions'])}")
        if core['primary_objects']:
            lines.append(f"- Objects: {', '.join(core['primary_objects'])}")
        if core['instruments']:
            lines.append(f"- Instruments: {', '.join(core['instruments'])}")
        if core['source_locations']:
            lines.append(f"- From: {', '.join(core['source_locations'])}")
        if core['target_locations']:
            lines.append(f"- To: {', '.join(core['target_locations'])}")
        lines.append(f"- Time: Actions {time_info['action_indices']} (duration: {time_info['duration']})")
        if disc_terms:
            lines.append(f"- Key terms: {', '.join(disc_terms)}")
        lines.append("")
        
        lines.append("**Actions in this episode:**")
        for idx in time_info["action_indices"]:
            verb = get_action_verb(action_sequence[idx])
            triplets = " ".join(str(t) for t in action_sequence[idx])
            lines.append(f"- Action {idx} ({verb}): {triplets}")
        lines.append("")
    
    return "\n".join(lines)

# Generate formatted timeline
timeline = format_timeline(hierarchy, action_sequence)
print(timeline)

## Overall Goal
Activity sequence

## Activity Timeline

### Episode 0: Episode 0
Actions: pick-up, sweep, close, place, open, put

**Structure:**
- Agent: person
- Actions: pick-up, sweep, close, place, open, put
- Objects: mop-stick, floor, door, cloth
- Instruments: hand1, hand2, mop-stick
- From: floor
- To: car, floor
- Time: Actions [0, 1, 2, 3, 4, 5] (duration: 6)
- Key terms: mop-stick, floor, door, cloth, car

**Actions in this episode:**
- Action 0 (pick-up): ['pick-up', 'with', 'hand1'] ['pick-up', 'with', 'hand2'] ['mop-stick', 'from', 'floor'] ['person', 'verb', 'pick-up'] ['pick-up', 'dobj', 'mop-stick']
- Action 1 (sweep): ['sweep', 'with', 'hand1'] ['sweep', 'with', 'hand2'] ['sweep', 'with', 'mop-stick'] ['sweep', 'dobj', 'floor'] ['sweep', 'in', 'car'] ['person', 'verb', 'sweep']
- Action 2 (close): ['close', 'with', 'hand1'] ['close', 'dobj', 'door'] ['person', 'verb', 'close']
- Action 3 (place): ['place', 'with', 'hand2'] ['place', 'dobj', 'mop-stick'] ['place', 'o

## Summary

In this tutorial, we demonstrated the complete EpiMine pipeline:

1. **Load data**: Scene graph triplets representing actions
2. **Build background**: All actions from dataset for salience computation
3. **Compute salience**: Identify discriminative terms using TF-IDF-like formula
4. **Extract key terms**: Filter by `min_freq` threshold
5. **Build co-occurrence matrix**: Jaccard similarity between actions
6. **Detect boundaries**: `mean - 1σ` threshold on consecutive scores
7. **Build structured episodes**: Rich format with temporal context
8. **Format for QA**: Human-readable timeline

### Key Parameters

| Parameter | Default | Effect |
|-----------|---------|--------|
| `min_freq` | 2 | Filter rare terms |
| `threshold_std` | 1.0 | Boundary sensitivity |

### What's NOT Used in SGQA (vs Original EpiMine)

- **Term expansion (cosine ≥ 0.75)**: Not needed — fixed vocabulary, no synonyms
- **Cross-document clustering**: Not needed — single sample processing
- **Embedding similarity**: Not needed — triplets are structured